In [1]:
import os
import datetime

import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "org.apache.hudi:hudi-spark3.3-bundle_2.12:0.12.0") \
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

In [ ]:
spark = builder.getOrCreate()
# It might take briefly to start the Spark process.
# Wait for some output below to let the Spark fully started.

In [ ]:
spark

In [5]:
tblname = "tripstbl"
tblpath = "file://" + os.getcwd() + "/out/tripstbl"
sc = spark.sparkContext

In [6]:
# generate Trips data
data_gen = sc._jvm.org.apache.hudi.QuickstartUtils.DataGenerator()
data_inserts = sc._jvm.org.apache.hudi.QuickstartUtils.convertToStringList(data_gen.generateInserts(10))

In [7]:
data_inserts

['{"ts": 1663432923115, "uuid": "ae4c3ed5-d4b7-48ff-94a7-af420eb9eeb1", "rider": "rider-213", "driver": "driver-213", "begin_lat": 0.4726905879569653, "begin_lon": 0.46157858450465483, "end_lat": 0.754803407008858, "end_lon": 0.9671159942018241, "fare": 34.158284716382845, "partitionpath": "americas/brazil/sao_paulo"}', '{"ts": 1663441613239, "uuid": "cb64f561-b013-4e12-b951-89d2a3fabd18", "rider": "rider-213", "driver": "driver-213", "begin_lat": 0.6100070562136587, "begin_lon": 0.8779402295427752, "end_lat": 0.3407870505929602, "end_lon": 0.5030798142293655, "fare": 43.4923811219014, "partitionpath": "americas/brazil/sao_paulo"}', '{"ts": 1663271105357, "uuid": "5ec24811-45cf-42f0-af44-ab1569b41e7e", "rider": "rider-213", "driver": "driver-213", "begin_lat": 0.5731835407930634, "begin_lon": 0.4923479652912024, "end_lat": 0.08988581780930216, "end_lon": 0.42520899698713666, "fare": 64.27696295884016, "partitionpath": "americas/united_states/san_francisco"}', '{"ts": 1662978858494, "uu

In [8]:
type(data_inserts)

py4j.java_collections.JavaList

In [ ]:
# read back into Python PySpark dataframe
df = spark.read.json(sc.parallelize(data_inserts, 2))
# wait for moment to let Spark process finish
# should observe loading progress bar below as cell output

In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
df

DataFrame[begin_lat: double, begin_lon: double, driver: string, end_lat: double, end_lon: double, fare: double, partitionpath: string, rider: string, ts: bigint, uuid: string]

In [12]:
df.printSchema()

root
 |-- begin_lat: double (nullable = true)
 |-- begin_lon: double (nullable = true)
 |-- driver: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lon: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- partitionpath: string (nullable = true)
 |-- rider: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- uuid: string (nullable = true)



In [13]:
df.show()

+-------------------+-------------------+----------+-------------------+-------------------+------------------+--------------------+---------+-------------+--------------------+
|          begin_lat|          begin_lon|    driver|            end_lat|            end_lon|              fare|       partitionpath|    rider|           ts|                uuid|
+-------------------+-------------------+----------+-------------------+-------------------+------------------+--------------------+---------+-------------+--------------------+
| 0.4726905879569653|0.46157858450465483|driver-213|  0.754803407008858| 0.9671159942018241|34.158284716382845|americas/brazil/s...|rider-213|1663432923115|ae4c3ed5-d4b7-48f...|
| 0.6100070562136587| 0.8779402295427752|driver-213| 0.3407870505929602| 0.5030798142293655|  43.4923811219014|americas/brazil/s...|rider-213|1663441613239|cb64f561-b013-4e1...|
| 0.5731835407930634| 0.4923479652912024|driver-213|0.08988581780930216|0.42520899698713666| 64.27696295884016

In [14]:
# now let write it out this in-memory dataframe into Hudi table
# in the following, we will do insert and update a Hudi table of default table type: Copy on Write
# https://hudi.apache.org/docs/table_types/#copy-on-write-table

# observe that how we use raw data fields "uuid", "ts", "partitionpath"
# map to table options for those Hudi table mandatory keys
# https://hudi.apache.org/docs/writing_data/

hudi_options = {
    'hoodie.table.name': tblname,
    'hoodie.datasource.write.recordkey.field': 'uuid',
    'hoodie.datasource.write.partitionpath.field': 'partitionpath',
    'hoodie.datasource.write.table.name': tblname,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field': 'ts',
    'hoodie.upsert.shuffle.parallelism': 2,
    'hoodie.insert.shuffle.parallelism': 2
}

In [ ]:
df.write.format("hudi").options(**hudi_options).mode("overwrite").save(tblpath)
# wait for moment to let Spark process finish
# should observe loading progress bar below as cell output

In [ ]:
# should see 'tripstbl' folder created within 'out' directory
!ls -al out/

In [17]:
# now let read back Hudi table
trips_df = spark.read.format("hudi").load(tblpath)

In [ ]:
trips_df

In [19]:
trips_df.printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- begin_lat: double (nullable = true)
 |-- begin_lon: double (nullable = true)
 |-- driver: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lon: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- rider: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- uuid: string (nullable = true)
 |-- partitionpath: string (nullable = true)



In [20]:
trips_df.show(2)

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------------+-------------------+----------+------------------+------------------+-----------------+---------+-------------+--------------------+--------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|          begin_lat|          begin_lon|    driver|           end_lat|           end_lon|             fare|    rider|           ts|                uuid|       partitionpath|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------------+-------------------+----------+------------------+------------------+-----------------+---------+-------------+--------------------+--------------------+
|  20220919101243481|20220919101243481...|d7ffede2-cbe4-4c8...|  americas/united_s...|5ce94bba-1c4c-48e...|0.21624150367601136|0.14285051259466197|driver-213

In [21]:
trips_df.dtypes

[('_hoodie_commit_time', 'string'),
 ('_hoodie_commit_seqno', 'string'),
 ('_hoodie_record_key', 'string'),
 ('_hoodie_partition_path', 'string'),
 ('_hoodie_file_name', 'string'),
 ('begin_lat', 'double'),
 ('begin_lon', 'double'),
 ('driver', 'string'),
 ('end_lat', 'double'),
 ('end_lon', 'double'),
 ('fare', 'double'),
 ('rider', 'string'),
 ('ts', 'bigint'),
 ('uuid', 'string'),
 ('partitionpath', 'string')]

In [22]:
# create table snapshot view to query with spark.sql(...)
trips_df.createOrReplaceTempView("trips_snapshot")

In [23]:
spark.sql("SELECT COUNT(*) FROM trips_snapshot").show()

+--------+
|count(1)|
+--------+
|      10|
+--------+



In [24]:
spark.sql("SELECT fare, begin_lon, begin_lat, ts FROM trips_snapshot WHERE fare > 20.0").show()

+------------------+-------------------+-------------------+-------------+
|              fare|          begin_lon|          begin_lat|           ts|
+------------------+-------------------+-------------------+-------------+
| 93.56018115236618|0.14285051259466197|0.21624150367601136|1662978858494|
| 27.79478688582596| 0.6273212202489661|0.11488393157088261|1663507472596|
| 33.92216483948643| 0.9694586417848392| 0.1856488085068272|1663335845880|
| 64.27696295884016| 0.4923479652912024| 0.5731835407930634|1663271105357|
|  43.4923811219014| 0.8779402295427752| 0.6100070562136587|1663441613239|
|34.158284716382845|0.46157858450465483| 0.4726905879569653|1663432923115|
| 66.62084366450246|0.03844104444445928| 0.0750588760043035|1663413757285|
| 41.06290929046368| 0.8192868687714224|  0.651058505660742|1663409926472|
+------------------+-------------------+-------------------+-------------+



In [25]:
spark.sql("SELECT _hoodie_commit_time, _hoodie_record_key, _hoodie_partition_path, rider, driver, fare FROM trips_snapshot").show()

+-------------------+--------------------+----------------------+---------+----------+------------------+
|_hoodie_commit_time|  _hoodie_record_key|_hoodie_partition_path|    rider|    driver|              fare|
+-------------------+--------------------+----------------------+---------+----------+------------------+
|  20220919101243481|d7ffede2-cbe4-4c8...|  americas/united_s...|rider-213|driver-213| 93.56018115236618|
|  20220919101243481|0c74c29c-d53f-484...|  americas/united_s...|rider-213|driver-213| 27.79478688582596|
|  20220919101243481|30c19156-4e9a-413...|  americas/united_s...|rider-213|driver-213| 33.92216483948643|
|  20220919101243481|5ec24811-45cf-42f...|  americas/united_s...|rider-213|driver-213| 64.27696295884016|
|  20220919101243481|a4f6cc61-e81f-463...|  americas/united_s...|rider-213|driver-213|19.179139106643607|
|  20220919101243481|cb64f561-b013-4e1...|  americas/brazil/s...|rider-213|driver-213|  43.4923811219014|
|  20220919101243481|ae4c3ed5-d4b7-48f...|  am

In [26]:
cut_off_ts_row = spark.sql("SELECT ts FROM trips_snapshot ORDER BY ts DESC LIMIT 1").collect()[0]
# type(cut_off_ts_row)
cut_off_ts = cut_off_ts_row['ts']
cut_off_ts

1663507472596

In [30]:
cut_off_dt = datetime.datetime.fromtimestamp(cut_off_ts / 1000.0)
cut_off_dt = cut_off_dt + datetime.timedelta(days=2)
cut_off_dt_str = cut_off_dt.strftime('%Y-%m-%d')
cut_off_dt_str

'2022-09-20'

In [31]:
# time travel query
# trips_tt_df = spark.read.format("hudi").option("as.of.instant", "20210728141108").load(tblpath)

# alternate time formats
# trips_tt_df = spark.read.format("hudi").option("as.of.instant", "2021-07-28 14:11:08").load(tblpath)

# it is equal to "as.of.instant = 2021-07-28 00:00:00"
# trips_tt_df = spark.read.format("hudi").option("as.of.instant", "2021-07-28").load(tblpath)

trips_tt_df = spark.read.format("hudi").option("as.of.instant", cut_off_dt_str).load(tblpath)

In [32]:
trips_tt_df.count()

10

In [33]:
# Update data
snapshot_query = "SELECT begin_lat, begin_lon, driver, end_lat, end_lon, fare, partitionpath, rider, ts, uuid FROM trips_snapshot"

In [34]:
snapshot_before_update = spark.sql(snapshot_query)
snapshot_before_update.count()

10

In [35]:
data_updates = sc._jvm.org.apache.hudi.QuickstartUtils.convertToStringList(data_gen.generateUpdates(10))

In [36]:
data_updates

['{"ts": 1663537086521, "uuid": "5ec24811-45cf-42f0-af44-ab1569b41e7e", "rider": "rider-284", "driver": "driver-284", "begin_lat": 0.7340133901254792, "begin_lon": 0.5142184937933181, "end_lat": 0.7814655558162802, "end_lon": 0.6592596683641996, "fare": 49.527694252432056, "partitionpath": "americas/united_states/san_francisco"}', '{"ts": 1663469311600, "uuid": "ae4c3ed5-d4b7-48ff-94a7-af420eb9eeb1", "rider": "rider-284", "driver": "driver-284", "begin_lat": 0.1593867607188556, "begin_lon": 0.010872312870502165, "end_lat": 0.9808530350038475, "end_lon": 0.7963756520507014, "fare": 29.47661370147079, "partitionpath": "americas/brazil/sao_paulo"}', '{"ts": 1663010453495, "uuid": "ae4c3ed5-d4b7-48ff-94a7-af420eb9eeb1", "rider": "rider-284", "driver": "driver-284", "begin_lat": 0.7180196467760873, "begin_lon": 0.13755354862499358, "end_lat": 0.3037264771699937, "end_lon": 0.2539047155055727, "fare": 86.75932789048282, "partitionpath": "americas/brazil/sao_paulo"}', '{"ts": 1663468536672, "

In [37]:
df = spark.read.json(sc.parallelize(data_updates, 2))

In [38]:
df

DataFrame[begin_lat: double, begin_lon: double, driver: string, end_lat: double, end_lon: double, fare: double, partitionpath: string, rider: string, ts: bigint, uuid: string]

In [39]:
df.count()

10

In [ ]:
df.write.format("hudi").options(**hudi_options).mode("append").save(tblpath)
# wait for moment to let Spark process finish
# should observe loading progress bar below as cell output

In [41]:
# validations
assert spark.sql(snapshot_query).count() == 10
assert df.count() == 10

In [42]:
# intersect might be non-deterministic as it depends on how/when 'data_updates' generated with what randomness
# observe how many data points get intersect after updated
spark.sql(snapshot_query).intersect(df).count()

0

In [43]:
# validations
assert spark.sql(snapshot_query).intersect(df).count() == 0

In [44]:
# re-load 'trips_df' from current table state
trips_df = spark.read.format("hudi").load(tblpath)

In [45]:
trips_df.count()

10

In [46]:
trips_df.printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- begin_lat: double (nullable = true)
 |-- begin_lon: double (nullable = true)
 |-- driver: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lon: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- rider: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- uuid: string (nullable = true)
 |-- partitionpath: string (nullable = true)



In [47]:
trips_df.createOrReplaceTempView("trips_updated")

In [50]:
spark.sql("SELECT uuid, ts, partitionpath, rider, driver, fare, begin_lon, begin_lat, end_lon, end_lat FROM trips_snapshot ORDER BY uuid DESC").show()

+--------------------+-------------+--------------------+---------+----------+------------------+-------------------+-------------------+-------------------+-------------------+
|                uuid|           ts|       partitionpath|    rider|    driver|              fare|          begin_lon|          begin_lat|            end_lon|            end_lat|
+--------------------+-------------+--------------------+---------+----------+------------------+-------------------+-------------------+-------------------+-------------------+
|e782c951-458b-4d3...|1663239335081|  asia/india/chennai|rider-213|driver-213|17.851135255091155| 0.5644092139040959|   0.40613510977307|0.02698359227182834|  0.798706304941517|
|d7ffede2-cbe4-4c8...|1662978858494|americas/united_s...|rider-213|driver-213| 93.56018115236618|0.14285051259466197|0.21624150367601136| 0.0966823831927115| 0.5890949624813784|
|cb64f561-b013-4e1...|1663441613239|americas/brazil/s...|rider-213|driver-213|  43.4923811219014| 0.8779402295

In [51]:
spark.sql("SELECT uuid, ts, partitionpath, rider, driver, fare, begin_lon, begin_lat, end_lon, end_lat FROM trips_updated ORDER BY uuid DESC").show()

+--------------------+-------------+--------------------+---------+----------+------------------+--------------------+--------------------+-------------------+------------------+
|                uuid|           ts|       partitionpath|    rider|    driver|              fare|           begin_lon|           begin_lat|            end_lon|           end_lat|
+--------------------+-------------+--------------------+---------+----------+------------------+--------------------+--------------------+-------------------+------------------+
|e782c951-458b-4d3...|1663445419934|  asia/india/chennai|rider-284|driver-284| 90.25710109008239|  0.4006983139989222| 0.08528650347654165|  0.908216792146506|0.1975324518739051|
|d7ffede2-cbe4-4c8...|1662978858494|americas/united_s...|rider-213|driver-213| 93.56018115236618| 0.14285051259466197| 0.21624150367601136| 0.0966823831927115|0.5890949624813784|
|cb64f561-b013-4e1...|1663441613239|americas/brazil/s...|rider-213|driver-213|  43.4923811219014|  0.8779